# Verificando as variaveis de contexto do spark

In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [2]:
spark

# Fazendo a leitura dos arquivos de log da nasa

In [3]:
logJulRdd = sc.textFile("C:\\Users\\Matheus\\Documents\\nasa\\access_log_Jul95")
logAgoRdd = sc.textFile("C:\\Users\\Matheus\\Documents\\nasa\\access_log_Aug95")

In [4]:
headerLog = ["hostname", "dataRequisicao", "requisicao", "codigoRetorno", "totalBytes"]

#  Função responsável por separar linhas do log

In [5]:
def separadorDeLinha(linha):
    
    try:
        #Criando variavel responsavel por manipular minha linha
        linhaAux = linha
        #Capturando Hostname
        hostname = linha[:linha.index('[')].strip().split(" ")[0]
        #Capturando timestamp
        comecoTimestamp = linhaAux.index("[") + 1
        fimTimestamp = linhaAux.index("]")
        timestamp = linhaAux[comecoTimestamp:fimTimestamp].strip()
        timestamp = timestamp[:11]
        #Manipulando linha original, para trabalhar com uma linha menor, ignorando a parte que já foi capturada!
        linhaAux = linhaAux[fimTimestamp + 1:]
        #Capturando requisicao
        comecoRequisicao = linhaAux.index('"') + 1
        linhaAux = linhaAux[comecoRequisicao:]
        linhaContrario = linhaAux[::-1]
        fimRequisicao = len(linhaAux) - linhaContrario.index('"') - 1
        requisicao = linhaAux[:fimRequisicao].strip()
        linhaAux = linhaAux[fimRequisicao + 1:]
        #Como manipulei minha linha, sobraram agora apenas o codigo retorno e o totalBytes, que estão separados por espaço
        linhaAux = linhaAux.strip()
        arrayCodigoETotalBytes = linhaAux.strip().split(" ")
        #Obtendo codigo retorno
        codigoRetorno = arrayCodigoETotalBytes[0]
        
        #Obtendo total de bytes
    except:
        return "Error" 
    try:
        totalBytes = arrayCodigoETotalBytes[1]
    except:
        totalBytes = "0"
    return hostname + ";"+ timestamp + ";" + requisicao + ";" + codigoRetorno + ";" + totalBytes    

# Construção do DataFrame

In [6]:

bodyJul = logJulRdd.map(lambda x: separadorDeLinha(x)).filter(lambda x: x != "Error")
bodyAgo = logAgoRdd.map(lambda x: separadorDeLinha(x)).filter(lambda x: x != "Error")

In [7]:
dfLogJul = bodyJul.map(lambda x: x.split(';')).filter(lambda x: len(x) == 5).toDF(headerLog)
dfLogAgo = bodyAgo.map(lambda x: x.split(';')).filter(lambda x: len(x) == 5).toDF(headerLog)

In [8]:

dfLogUniao = dfLogJul.union(dfLogAgo)
dfLogUniao = dfLogUniao.selectExpr("hostname", "dataRequisicao", "requisicao", "cast(codigoRetorno as int)", "cast(totalBytes as float)")

In [9]:

from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)


# Resolvendo exercicios propostos no desafio

In [13]:
dfLogUniao.createOrReplaceTempView("LogNasa")



+--------------------+--------------+--------------------+-------------+----------+
|            hostname|dataRequisicao|          requisicao|codigoRetorno|totalBytes|
+--------------------+--------------+--------------------+-------------+----------+
|        199.72.81.55|   01/Jul/1995|GET /history/apol...|          200|    6245.0|
|unicomp6.unicomp.net|   01/Jul/1995|GET /shuttle/coun...|          200|    3985.0|
|      199.120.110.21|   01/Jul/1995|GET /shuttle/miss...|          200|    4085.0|
|  burger.letters.com|   01/Jul/1995|GET /shuttle/coun...|          304|       0.0|
|      199.120.110.21|   01/Jul/1995|GET /shuttle/miss...|          200|    4179.0|
|  burger.letters.com|   01/Jul/1995|GET /images/NASA-...|          304|       0.0|
|  burger.letters.com|   01/Jul/1995|GET /shuttle/coun...|          200|       0.0|
|     205.212.115.106|   01/Jul/1995|GET /shuttle/coun...|          200|    3985.0|
|         d104.aa.net|   01/Jul/1995|GET /shuttle/coun...|          200|    

# Iniciando as respostas às questões do desafio

Número de hosts únicos

In [12]:
sqlContext.sql("select count(distinct(hostname)) from LogNasa").show()

+------------------------+
|count(DISTINCT hostname)|
+------------------------+
|                  137978|
+------------------------+



O total de erros 404

In [14]:
sqlContext.sql("select count(codigoRetorno) from LogNasa where codigoRetorno = 404").show()

+--------------------+
|count(codigoRetorno)|
+--------------------+
|               20895|
+--------------------+



Os 5 URLs que causaram mais erros 404

In [15]:
sqlContext.sql("select hostname, count(codigoRetorno) from LogNasa where codigoRetorno = 404 group by hostname order by count(codigoRetorno) desc limit 5").show(truncate=False)

+---------------------------+--------------------+
|hostname                   |count(codigoRetorno)|
+---------------------------+--------------------+
|hoohoo.ncsa.uiuc.edu       |251                 |
|piweba3y.prodigy.com       |155                 |
|jbiagioni.npt.nuwc.navy.mil|132                 |
|piweba1y.prodigy.com       |114                 |
|www-d4.proxy.aol.com       |91                  |
+---------------------------+--------------------+



Quantidade de erros 404 por dia

In [26]:
dfErrosPorDia = sqlContext.sql("select dataRequisicao, count(codigoRetorno) as totalCodigoRetorno from LogNasa where codigoRetorno = 404 group by dataRequisicao order by dataRequisicao")

In [27]:
dfErrosPorDia.createOrReplaceTempView("erros")

In [29]:
sqlContext.sql("select avg(totalCodigoRetorno) from erros").show()

+-----------------------+
|avg(totalCodigoRetorno)|
+-----------------------+
|      360.2586206896552|
+-----------------------+



Total de bytes

In [32]:
sqlContext.sql("select sum(totalBytes) from LogNasa").show()

+---------------+
|sum(totalBytes)|
+---------------+
|6.5524314098E10|
+---------------+

